In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_score, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import manifold, decomposition, linear_model, ensemble, neighbors, cross_validation, metrics


/Users/Roman/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv("train_without_oversampling.csv")
test = pd.read_csv("test.csv")

y = train.target
train.drop("target", axis=1, inplace=True)

train.drop("client_id", axis=1, inplace=True)
ans = test[["client_id"]]
test.drop("client_id", axis=1, inplace=True)


In [3]:
train_data, test_data, y_train, y_test = train_test_split(train, y, test_size = 0.3, stratify = y)

In [7]:
lin_cl = linear_model.LogisticRegression(class_weight="balanced", verbose=1, n_jobs=-1, C=0.8)
lin_cl.fit(train_data, y_train)
preds = lin_cl.predict(test_data)
res = metrics.roc_auc_score(y_test, preds)
print res

[LibLinear]0.598255489005


In [8]:
rf = ensemble.RandomForestClassifier(n_estimators=900, n_jobs=-1, max_depth=30 ,class_weight='balanced')
rf.fit(train_data, y_train)
preds = rf.predict(test_data)
res = metrics.roc_auc_score(y_test, preds)
print res

0.535437556119


In [ ]:
from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [ ]:
def objective(space):
    print space
    c = space['C']
    pen = space['pen']
 
    cv = cross_validation.StratifiedShuffleSplit(y, n_iter=5, test_size=0.3)
    lin_cl = linear_model.LogisticRegression(C=c, penalty=pen, n_jobs=-1)
    res = cross_validation.cross_val_score(estimator=lin_cl, X=train, y=y, scoring="roc_auc", \
                                    cv = cv, n_jobs=-1)
    print res.mean()
    return{'loss': -res.mean(), 'status': STATUS_OK}


space ={
        'C' : hp.uniform('C', 0.1, 2.0),
        'pen' : hp.choice('pen', ["l1", "l2"])
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=75,
            trials=trials)

print best

best 
{'C': 0.34394279435077646, 'pen': 'l1'}
0.605996210639

In [ ]:
print best

In [5]:
y_ans = lin_cl.predict(test)
ans['val'] = y_ans
ans.rename(columns={"client_id":"_ID_", "val":"_VAL_"}, inplace=True)
ans.to_csv("linear_classification.csv", index=False, sep=",", header=False)

/Users/Roman/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/Roman/anaconda/lib/python2.7/site-packages/pandas/core/frame.py:2816: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)
